The Long Short-Term Memory or LSTM network is a recurrent neural network that is trained using Backpropagation Through Time and overcomes the vanishing gradient problem.

As such it can be used to create large recurrent networks, that in turn can be used to address difficult sequence problems in machine learning and achieve state-of-the-art results.

Instead of neurons, LSTM networks have memory blocks that are connected into layers.

A block has components that make it smarter than a classical neuron and a memory for recent sequences. A block contains gates that manage the block’s state and output. A block operates upon an input sequence and each gate within a block uses the sigmoid activation units to control whether they are triggered or not, making the change of state and addition of information flowing through the block conditional.

There are three types of gates within a unit:

    Forget Gate: conditionally decides what information to throw away from the block.
    Input Gate: conditionally decides which values from the input to update the memory state.
    Output Gate: conditionally decides what to output based on input and the memory of the block.

Each unit is like a mini-state machine where the gates of the units have weights that are learned during the training procedure.

You can see how you may achieve a sophisticated learning and memory from a layer of LSTMs, and it is not hard to imagine how higher-order abstractions may be layered with multiple such layers.
LSTM Network For Regression

We can phrase the problem as a regression problem.

That is, given the number of passengers (in units of thousands) this month, what is the number of passengers next month.

We can write a simple function to convert our single column of data into a two-column dataset. The first column containing this month’s (t) passenger count and the second column containing next month’s (t+1) passenger count, to be predicted.



In [1]:
import pandas
import matplotlib.pyplot as plt
dataset = pandas.read_csv('international-airline-passengers.csv', usecols=[1], engine='python', skipfooter=3)
plt.plot(dataset)
plt.show()

Before we get started, let’s first import all of the functions and classes we intend to use. This assumes a working SciPy environment with the Keras deep learning library installed.

In [52]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.grid_search import GridSearchCV

Before we do anything, it is a good idea to fix the random number seed to ensure our results are reproducible.

In [4]:
# fix random seed for reproducibility
numpy.random.seed(7)

We can also use the code from the previous section to load the dataset as a Pandas dataframe. We can then extract the NumPy array from the dataframe and convert the integer values to floating point values which are more suitable for modeling with a neural network.

In [5]:
# load the dataset
dataframe = pandas.read_csv('international-airline-passengers.csv', usecols=[1], engine='python', skipfooter=3)
dataset = dataframe.values
dataset = dataset.astype('float32')

LSTMs are sensitive to the scale of the input data, specifically when the sigmoid (default) or tanh activation functions are used. It can be a good practice to rescale the data to the range of 0-to-1, also called normalizing. We can easily normalize the dataset using the MinMaxScaler preprocessing class from the scikit-learn library.

In [6]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
print(dataset)

[[ 0.01544401]
 [ 0.02702703]
 [ 0.05405405]
 [ 0.04826255]
 [ 0.03281853]
 [ 0.05984557]
 [ 0.08494207]
 [ 0.08494207]
 [ 0.06177607]
 [ 0.02895753]
 [ 0.        ]
 [ 0.02702703]
 [ 0.02123553]
 [ 0.04247104]
 [ 0.07142857]
 [ 0.05984557]
 [ 0.04054055]
 [ 0.08687258]
 [ 0.12741312]
 [ 0.12741312]
 [ 0.10424709]
 [ 0.05598456]
 [ 0.01930502]
 [ 0.06949806]
 [ 0.07915059]
 [ 0.08880308]
 [ 0.14285713]
 [ 0.11389962]
 [ 0.13127413]
 [ 0.14285713]
 [ 0.18339768]
 [ 0.18339768]
 [ 0.15444016]
 [ 0.11196911]
 [ 0.08108109]
 [ 0.1196911 ]
 [ 0.12934363]
 [ 0.14671814]
 [ 0.17181468]
 [ 0.14864865]
 [ 0.15250966]
 [ 0.22007722]
 [ 0.24324325]
 [ 0.26640925]
 [ 0.2027027 ]
 [ 0.16795367]
 [ 0.13127413]
 [ 0.17374519]
 [ 0.17760617]
 [ 0.17760617]
 [ 0.25482625]
 [ 0.25289574]
 [ 0.24131274]
 [ 0.26833975]
 [ 0.3088803 ]
 [ 0.32432434]
 [ 0.25675675]
 [ 0.20656371]
 [ 0.14671814]
 [ 0.18725869]
 [ 0.19305018]
 [ 0.16216215]
 [ 0.25289574]
 [ 0.23745173]
 [ 0.25096524]
 [ 0.3088803 ]
 [ 0.38223

After we model our data and estimate the skill of our model on the training dataset, we need to get an idea of the skill of the model on new unseen data. For a normal classification or regression problem we would do this using cross validation.

With time series data, the sequence of values is important. A simple method that we can use is to split the ordered dataset into train and test datasets. The code below calculates the index of the split point and separates the data into the training datasets with 67% of the observations that we can use to train our model, leaving the remaining 33% for testing the model.

In [7]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))
print(dataset[0:2,0])
print(dataset[2,0])

(96, 48)
[ 0.01544401  0.02702703]
0.0540541


Now we can define a function to create a new dataset as described above.

The function takes two arguments, the dataset which is a NumPy array that we want to convert into a dataset and the look_back which is the number of previous time steps to use as input variables to predict the next time period, in this case, defaulted to 1.

This default will create a dataset where X is the number of passengers at a given time (t) and Y is the number of passengers at the next time (t + 1).

It can be configured and we will by constructing a differently shaped dataset in the next section.

In [8]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)

If you compare these first 5 rows to the original dataset sample listed in the previous section, you can see the X=t and Y=t+1 pattern in the numbers.

Let’s use this function to prepare the train and test datasets ready for modeling.

In [49]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

trainX.shape[1]

1

The LSTM network expects the input data (X) to be provided with a specific array structure in the form of: [samples, time steps, features].

Currently, our data is in the form: [samples, features] and we are framing the problem as one time step for each sample. We can transform the prepared train and test input data into the expected structure using numpy.reshape() as follows:

In [50]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

trainX.shape

(94, 1, 1)

We are now ready to design and fit our LSTM network for this problem.

The network has a visible layer with 1 input, a hidden layer with 4 LSTM blocks or neurons and an output layer that makes a single value prediction. The default sigmoid activation function is used for the LSTM blocks. The network is trained for 100 epochs and a batch size of 1 is used.

In [45]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_dim=look_back))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, nb_epoch=100, batch_size=1, verbose=2)

Epoch 1/100
0s - loss: 0.0524
Epoch 2/100
0s - loss: 0.0248
Epoch 3/100
0s - loss: 0.0164
Epoch 4/100
0s - loss: 0.0140
Epoch 5/100
0s - loss: 0.0128
Epoch 6/100
0s - loss: 0.0117
Epoch 7/100
0s - loss: 0.0107
Epoch 8/100
0s - loss: 0.0097
Epoch 9/100
0s - loss: 0.0088
Epoch 10/100
0s - loss: 0.0079
Epoch 11/100
0s - loss: 0.0072
Epoch 12/100
0s - loss: 0.0064
Epoch 13/100
0s - loss: 0.0059
Epoch 14/100
0s - loss: 0.0055
Epoch 15/100
0s - loss: 0.0051
Epoch 16/100
0s - loss: 0.0048
Epoch 17/100
0s - loss: 0.0045
Epoch 18/100
0s - loss: 0.0045
Epoch 19/100
0s - loss: 0.0042
Epoch 20/100
0s - loss: 0.0041
Epoch 21/100
0s - loss: 0.0040
Epoch 22/100
0s - loss: 0.0040
Epoch 23/100
0s - loss: 0.0039
Epoch 24/100
0s - loss: 0.0039
Epoch 25/100
0s - loss: 0.0038
Epoch 26/100
0s - loss: 0.0037
Epoch 27/100
0s - loss: 0.0037
Epoch 28/100
0s - loss: 0.0036
Epoch 29/100
0s - loss: 0.0036
Epoch 30/100
0s - loss: 0.0036
Epoch 31/100
0s - loss: 0.0035
Epoch 32/100
0s - loss: 0.0035
Epoch 33/100
0s -

In [54]:
#Grid search parameters

def create_model(neurons=1):

	# create and fit the LSTM network
	model = Sequential()
	model.add(LSTM(neurons, input_dim=look_back))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam',  metrics=['accuracy'])
	#model.fit(trainX, trainY, nb_epoch=100, batch_size=1, verbose=2)
	return model

model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [1, 2, 3, 4, 5]
epochs = [100]
neurons = [2, 3, 4, 5]


param_grid = dict(neurons=neurons,batch_size=batch_size, nb_epoch=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(trainX, trainY)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
for params, mean_score, scores in grid_result.grid_scores_:
    print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))
print(grid_result.best_params_['nb_epoch'])	
print(grid_result.best_params_['neurons'])
print(grid_result.best_params_['batch_size'])
print(look_back)


Best: 0.010638 using {'nb_epoch': 100, 'neurons': 2, 'batch_size': 3}
0.010417 (0.014731) with: {'nb_epoch': 100, 'neurons': 2, 'batch_size': 1}
0.010417 (0.014731) with: {'nb_epoch': 100, 'neurons': 3, 'batch_size': 1}
0.010417 (0.014731) with: {'nb_epoch': 100, 'neurons': 4, 'batch_size': 1}
0.010417 (0.014731) with: {'nb_epoch': 100, 'neurons': 5, 'batch_size': 1}
0.010417 (0.014731) with: {'nb_epoch': 100, 'neurons': 2, 'batch_size': 2}
0.010417 (0.014731) with: {'nb_epoch': 100, 'neurons': 3, 'batch_size': 2}
0.010417 (0.014731) with: {'nb_epoch': 100, 'neurons': 4, 'batch_size': 2}
0.010417 (0.014731) with: {'nb_epoch': 100, 'neurons': 5, 'batch_size': 2}
0.010417 (0.014731) with: {'nb_epoch': 100, 'neurons': 2, 'batch_size': 3}
0.010417 (0.014731) with: {'nb_epoch': 100, 'neurons': 3, 'batch_size': 3}
0.010417 (0.014731) with: {'nb_epoch': 100, 'neurons': 4, 'batch_size': 3}
0.010417 (0.014731) with: {'nb_epoch': 100, 'neurons': 5, 'batch_size': 3}
0.010417 (0.014731) with: {'nb

Once the model is fit, we can estimate the performance of the model on the train and test datasets. This will give us a point of comparison for new models.

Note that we invert the normalization using the same scaler object to get mean errors in squared units (thousands of passengers per month).

In [46]:
# Estimate model performance
trainScore = model.evaluate(trainX, trainY, verbose=0)
trainScore = math.sqrt(trainScore)
trainScore = scaler.inverse_transform(numpy.array([[trainScore]]))
print('Train Score: %.2f RMSE' % (trainScore))

testScore = model.evaluate(testX, testY, verbose=0)
testScore = math.sqrt(testScore)
testScore = scaler.inverse_transform(numpy.array([[testScore]]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 129.53 RMSE
Test Score: 182.27 RMSE


Finally, we can generate predictions using the model for both the train and test dataset to get a visual indication of the skill of the model.

Because of how the dataset was prepared, we must shift the predictions so that they aline on the x-axis with the original dataset. Once prepared, the data is plotted, showing the original dataset in blue, the predictions for the train dataset in green and the predictions on the unseen test dataset in red.

In [18]:

# generate predictions for training
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

# plot baseline and predictions
plt.plot(dataset)
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()